<a href="https://colab.research.google.com/github/alaeddinehamroun/Recommender-Systems/blob/main/Matrix_Factorization_for_Movie_Recommendations_SVD_%26_SVD%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Why surprise? primarily because it implements SVD++
# Surprise is a Python scikit for building and analyzing recommender systems that deal with explicit rating data.
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195797 sha256=cca8ef8d58ff4b8b2efa34fbc68983dfdc32bcfbe25719c7ef9b5e1ed738f3aa
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


# Matrix Factorization

Matrix factorization is a popular technique used in machine learning and recommender systems for reducing the dimensionality of a large matrix by decomposing it into two or more lower-dimensional matrices. In the context of recommender systems, the large matrix represents the user-item rating matrix, where each cell of the matrix corresponds to a user's rating of a particular item.

Matrix factorization algorithms seek to factorize this matrix into two matrices - one that describes the users and the other that describes the items - such that the product of these matrices approximates the original matrix. The factors in these matrices can be interpreted as user and item features, which can be used to predict missing ratings or recommend items to users.

There are several matrix factorization algorithms, including Singular Value Decomposition (SVD), Non-negative Matrix Factorization (NMF), and Alternating Least Squares (ALS). These algorithms differ in their approach to matrix factorization, with some focusing on minimizing the error between the predicted and actual ratings, while others incorporate regularization to prevent overfitting or bias.

Matrix factorization has been shown to be an effective technique for dealing with the sparsity and scalability challenges in recommender systems, making it a popular approach in industry and academia.

## Imports & utils

In [3]:
from surprise import Dataset # To load the dataset.
from surprise import Reader  # To parse a file containing ratings.
                             # Such a file is assumed to specify only one rating per line,
                             # and each line needs to respect the following structure:
                             # user; item; rating; [timestamp]
from surprise import SVD, SVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import KNNBaseline

from collections import defaultdict
import itertools

In [4]:
def getTopN(predictions, n=10, minimumRating=4.0):
    """
    Top N recommendations based on predictions
  
    Args:
        predictions:
        n:
        minimumRating: 
    Returns:
        dict: The top N rated movies for each userId
    """
    # Create a dictionary of empty lists
    topN = defaultdict(list)
    # Loop through each prediction tuple and append movieId and estimatedRating to the corresponding userId in topN
    for userId, movieId, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userId)].append((int(movieId), estimatedRating))
    # For each userId in topN, sort the list of movie ratings by the estimated rating in descending order
    # and keep only the top n rated movies.
    for userId, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userId)] = ratings[:n]

    return topN

In [5]:
def getPopularityRanks(data):
    """
    Compute the popularity ranks
    Params:
        data: list of ratings
    """
    ratings = defaultdict(int)
    rankings = defaultdict(int)
    for row in data:
      movieID = int(row[1])
      ratings[movieID] += 1
    rank = 1
    for movieID, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
        rankings[movieID] = rank
        rank += 1
    return rankings    

## Evaluation metrics

* **Metrics for evaluating model accuracy**
  * **RMSE**: Root Mean Squared Error. Lower values mean better accuracy.
  * **MAE**: Mean Absolute Error. Lower values mean better accuracy.
  
  
* **Metrics for evaluating top end recommendations**
    * **Hit Rate (HR)**: How often we are able to recommend a left-out rating. Higher is better.
    * **Cumulative Hit Rate (cHR)**: Hit rate, confined to ratings above a certain threshold. Higher is better.
    * **Average Reciprocal Hit Rank (ARHR)**: Hit rate that takes the ranking into account. Higher is better.
    * **Coverage**: Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
    * **Diversity**: 1-S, where S is the average similarity score between every possible pair of recommendations for a given user. Higher means more diverse.
    * **Novelty**: Average popularity rank of recommended items. Higher means more novel.

##### Recommender metrics utils:

In [6]:
# Hit Rate
def hitRate(topNPredicted, leftOutPredictions):
    """
    The hit rate for a set of top N recommendations and left out predictions
    
    Args:
        topNPredicted: Top 10 recommendations per user.
        lefOutPredictions: Predicted ratings for left-out set: this set is composed of one rating per user.
    
    Returns:
        float: The hit rate.
    """
    
    hits = 0
    total = 0
    
    # for each left-out rating
    for leftOut in leftOutPredictions:
        userId = leftOut[0]
        leftOutMovieId = leftOut[1]
        # is it in the predicted top 10 for this user?
        hit = False
        for movieId, predictedRating in topNPredicted[int(userId)]:
            if (int(leftOutMovieId) == int(movieId)):
                hit = True
                break
        if (hit):
            hits +=1
        total +=1
        
    return hits/total

In [7]:
def CumulativeHitRate(topNPredicted, leftOutPredictions, ratingCutoff=0):
    hits = 0
    total = 0

    # For each left-out rating
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        # Only look at ability to recommend things the users actually liked...
        if (actualRating >= ratingCutoff):
            # Is it in the predicted top 10 for this user?
            hit = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutMovieID) == movieID):
                    hit = True
                    break
            if (hit) :
                hits += 1

            total += 1

    # Compute overall precision
    return hits/total

In [8]:
def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summation = 0
    total = 0
    # For each left-out rating
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        # Is it in the predicted top N for this user?
        hitRank = 0
        rank = 0
        for movieID, predictedRating in topNPredicted[int(userID)]:
            rank = rank + 1
            if (int(leftOutMovieID) == movieID):
                hitRank = rank
                break
        if (hitRank > 0) :
            summation += 1.0 / hitRank

        total += 1

    return summation / total

In [9]:
def UserCoverage(topNPredicted, numUsers, ratingThreshold=0):
    hits = 0
    for userID in topNPredicted.keys():
        hit = False
        for movieID, predictedRating in topNPredicted[userID]:
            if (predictedRating >= ratingThreshold):
                hit = True
                break
        if (hit):
            hits += 1

    return hits / numUsers

In [10]:
def Diversity(topNPredicted, simsAlgo):
    n = 0
    total = 0
    simsMatrix = simsAlgo.compute_similarities()
    for userID in topNPredicted.keys():
        pairs = itertools.combinations(topNPredicted[userID], 2)
        for pair in pairs:
            movie1 = pair[0][0]
            movie2 = pair[1][0]
            innerID1 = simsAlgo.trainset.to_inner_iid(str(movie1))
            innerID2 = simsAlgo.trainset.to_inner_iid(str(movie2))
            similarity = simsMatrix[innerID1][innerID2]
            total += similarity
            n += 1

    S = total / n
    return (1-S)

In [11]:
def Novelty(topNPredicted, rankings):
    n = 0
    total = 0
    for userID in topNPredicted.keys():
        for rating in topNPredicted[userID]:
            movieID = rating[0]
            rank = rankings[movieID]
            total += rank
            n += 1
    return total / n

In [12]:
def evaluate(predictions, algo, train_loocv, test_loocv, bigTestSet, n, fullTrainSet, fbigTestSet, rankings):
    metrics = {}
    # Evaluating accuracy of model
    metrics["RMSE"] = accuracy.rmse(predictions, verbose=False)
    metrics["MAE"] = accuracy.mae(predictions, verbose=False)
    
    #Evaluating  top-N recommendations
    # Hit rate
    # Train model without left-out ratings
    algo.fit(train_loocv)
    # Predict ratings for left-out ratings only
    leftOutPredictions = algo.test(test_loocv)
    # Predict all ratings that are not in the training set
    allPredictions = algo.test(bigTestSet) 
    # Compute top N recs for each user
    topNpredicted = getTopN(allPredictions, n)
    metrics["HR"] = hitRate(topNPredicted, leftOutPredictions)
    
    # Cumulative hit rate
    metrics["cHR"] = CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0)

    # Average reciprocal hit rank
    metrics["ARHR"] = AverageReciprocalHitRank(topNPredicted, leftOutPredictions)
    
    # Coverage
    # Compute complete recommendations, no hold outs
    algo.fit(fullTrainSet)
    allPredictions = algo.test(fbigTestSet)
    topNpredicted = getTopN(allPredictions, n)
    metrics["Coverage"] = UserCoverage(topNPredicted, fullTrainSet.n_users, ratingThreshold=4.0)
    
    # Diversity
    # Compute item similarities so we can measure diversity later
    sim_options = {'name': 'pearson_baseline', 'user_based': False}
    simsAlgo = KNNBaseline(sim_options=sim_options)
    simsAlgo.fit(fullTrainSet)
    metrics["Diversity"] = Diversity(topNPredicted, simsAlgo)
    
    # Novelty
    metrics["Novelty"] = Novelty(topNPredicted, rankings)
    
    
    return metrics

## Data loading

In [13]:
data = Dataset.load_builtin(name='ml-1m', prompt=True)

Dataset ml-1m could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


In [44]:
# Standard train/test split. This will be used for the ratings prediction regression task
trainSet, testSet = train_test_split(data, test_size=.25, random_state=1)

## SVD

SVD decomposes a given matrix A into three matrices, U, S, and V, such that A = U * S * V^T, where U and V are orthogonal matrices, and S is a diagonal matrix with non-negative values on the diagonal, called the singular values.

The U and V matrices in SVD represent the left and right singular vectors of the matrix A, respectively. These matrices contain the basis vectors that span the columns of A and capture the most significant patterns in the data. The S matrix contains the singular values of A, which provide a measure of the importance of each singular vector.

In the context of recommender systems, SVD is used to decompose the user-item rating matrix into user and item matrices with lower dimensions. The lower-dimensional matrices are then used to predict missing ratings or recommend items to users. This is achieved by computing the dot product between the user and item matrices to obtain the predicted ratings.

SVD has been shown to be an effective technique for handling the sparsity and scalability challenges in recommender systems. However, SVD can be computationally expensive for large matrices, and it may suffer from overfitting if not regularized properly. Therefore, there are several variants of SVD, such as Truncated SVD and Regularized SVD, which mitigate these issues.

In [15]:
svd = SVD(random_state=10)

In [16]:
svd.fit(trainSet)

In [17]:
# List of predictions on testSet
predictions = svd.test(testSet)

In [ ]:
predictions

#### 1. Evaluating accuracy of model

In [19]:
print("\nEvaluating accuracy of model...")
print("RMSE: ", accuracy.rmse(predictions, verbose=False))
print("MAE: ", accuracy.mae(predictions, verbose=False))


Evaluating accuracy of model...
RMSE:  0.875453779882117
MAE:  0.687012485376433


In [20]:
import gc

del trainSet
del testSet
del predictions

gc.collect()

0

#### 2. Evaluating top-N recommendations

##### 2.1. Hit rate:
See how often we recommended a movie the user actually rated

    How to compute hit rate? 

    The inital dataset is a matrix (n_users x n_movies) of actual ratings and zeros (or NA) where. Each cell corresponds to a pair (user_id, movie_id).

    Firstly, we split the data by seting aside one rating per user for testing (test_loocv) using leave-one-out cross-validation (LOOCV). All the other user's movies will be in the training set (train_loocv).

    After training the model on the training set, predictions are made on the left-out movie in the test set. Then, all the missing ratings from the training set are predicted using the trained model.

    Next, the top N recommendations are computed for each user. For every left-out movie in the test set, if that movie is present in the top N recommendations for the user, that is considered a hit.

    Finally, the hit rate is calculated as the ratio of the number of hits to the total number of left-out movies.


In [21]:
from surprise.dataset import Trainset
# Leave-one-out cross validation split. This will be used for hit-rate prediction
# The split is performed by seting aside one rating per user for testing. All the other (user, movie) combos will be in the trainset
# n_splits: number of folds
LOOCV = LeaveOneOut(n_splits=1, random_state=1) # Cross-validation iterator where each user has exactly one rating in the testset.

train_loocv, test_loocv = list(LOOCV.split(data))[0]

In [24]:
# Train model without left-out ratings
svd.fit(train_loocv)
# Predict ratings for left-out ratings only
leftOutPredictions = svd.test(test_loocv)

In [25]:
# Build predictions for all ratings not in the training set
# build_anti_testset: returns a list of ratings that are missing in the train_loocv 
bigTestSet = train_loocv.build_anti_testset()
allPredictions = svd.test(bigTestSet)

In [26]:
# Compute top 10 recs for each user
topNPredicted = getTopN(allPredictions, n=10)
# See how often we recommended a movie the user actually rated
print("\nHit Rate: ", hitRate(topNPredicted, leftOutPredictions))



Hit Rate:  0.03360927152317881


In [27]:
import gc
del bigTestSet
del train_loocv
del test_loocv
del allPredictions
gc.collect()

0

##### 2.2. Cumulative Hit Rate: 
see how often we recommended a movie the user actually liked

    Hit rate, confined to ratings above a certain threshold. 

In [28]:
# See how often we recommended a movie the user actually rated
print("\ncHR (Cumulative Hit Rate, rating >= 4): ", CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))



cHR (Cumulative Hit Rate, rating >= 4):  0.0509571313022378


##### 2.3. Average Reciprocal Hit Rank:
    Hit rate that takes the ranking into account.


In [29]:
# Compute ARHR
print("\nARHR (Average Reciprocal Hit Rank): ", AverageReciprocalHitRank(topNPredicted, leftOutPredictions))


ARHR (Average Reciprocal Hit Rank):  0.012887036161042793


In [30]:
import gc
del topNPredicted
del leftOutPredictions
gc.collect()

0

##### 2.4. Coverage:
    What percentage of users have at least one "good" recommendation


In [ ]:
# Prepare full dataset for training
fullTrainSet = data.build_full_trainset()

In [33]:
print("\nComputing complete recommendations, no hold outs...")
svd.fit(fullTrainSet)
bigTestSet = fullTrainSet.build_anti_testset()
allPredictions = svd.test(bigTestSet)
topNPredicted = getTopN(allPredictions, n=10)


Computing complete recommendations, no hold outs...


In [34]:
# Print user coverage with a minimum predicted rating of 4.0:
print("\nUser coverage: ", UserCoverage(topNPredicted, fullTrainSet.n_users, ratingThreshold=4.0))



User coverage:  0.9897350993377484


##### 2.5. Diversity

In [32]:
print("\nComputing item similarities so we can measure diversity later...")
sim_options = {'name': 'pearson_baseline', 'user_based': False}
simsAlgo = KNNBaseline(sim_options=sim_options)
simsAlgo.fit(fullTrainSet)


Computing item similarities so we can measure diversity later...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [35]:
# Measure diversity of recommendations:
print("\nDiversity: ", Diversity(topNPredicted, simsAlgo))

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Diversity:  0.933834367587023


##### 2.6. Novelty

In [37]:
import gc
del fullTrainSet
del allPredictions
del bigTestSet
gc.collect()

60

In [38]:
# Compute movie popularity ranks so we can measure novelty later
rankings = getPopularityRanks(data.raw_ratings)

In [39]:
# Measure novelty (average popularity rank of recommendations):
print("\nNovelty (average popularity rank): ", Novelty(topNPredicted, rankings))


Novelty (average popularity rank):  643.175474968562


In [42]:
import gc
del topNPredicted
del rankings
gc.collect()

128

## SVD++

The SVD++ algorithm, an extension of SVD taking into account implicit ratings.

In [43]:
svdpp = SVDpp(random_state=10)

In [45]:
svdpp.fit(trainSet)

In [46]:
# List of predictions on testSet
predictions = svdpp.test(testSet)

#### 1. Evaluating accuracy of model

In [ ]:
print("\nEvaluating accuracy of model...")
print("RMSE: ", accuracy.rmse(predictions, verbose=False))
print("MAE: ", accuracy.mae(predictions, verbose=False))


Evaluating accuracy of model...
RMSE:  0.865609967139632
MAE:  0.6755024733708137


In [ ]:
import gc

del trainSet
del testSet
del predictions

gc.collect()

0

#### 2. Evaluating top-N recommendations

##### 2.1. Hit rate:
See how often we recommended a movie the user actually rated

    How to compute hit rate? 

    The inital dataset is a matrix (n_users x n_movies) of actual ratings and zeros (or NA) where. Each cell corresponds to a pair (user_id, movie_id).

    Firstly, we split the data by seting aside one rating per user for testing (test_loocv) using leave-one-out cross-validation (LOOCV). All the other user's movies will be in the training set (train_loocv).

    After training the model on the training set, predictions are made on the left-out movie in the test set. Then, all the missing ratings from the training set are predicted using the trained model.

    Next, the top N recommendations are computed for each user. For every left-out movie in the test set, if that movie is present in the top N recommendations for the user, that is considered a hit.

    Finally, the hit rate is calculated as the ratio of the number of hits to the total number of left-out movies.


In [ ]:
from surprise.dataset import Trainset
# Leave-one-out cross validation split. This will be used for hit-rate prediction
# The split is performed by seting aside one rating per user for testing. All the other (user, movie) combos will be in the trainset
# n_splits: number of folds
LOOCV = LeaveOneOut(n_splits=1, random_state=1) # Cross-validation iterator where each user has exactly one rating in the testset.

train_loocv, test_loocv = list(LOOCV.split(data))[0]

In [ ]:
# Train model without left-out ratings
svdpp.fit(train_loocv)
# Predict ratings for left-out ratings only
leftOutPredictions = svdpp.test(test_loocv)

In [ ]:
# Build predictions for all ratings not in the training set
# build_anti_testset: returns a list of ratings that are missing in the train_loocv 
bigTestSet = train_loocv.build_anti_testset()
allPredictions = svdpp.test(bigTestSet)

In [ ]:
# Compute top 10 recs for each user
topNPredicted = getTopN(allPredictions, n=10)
# See how often we recommended a movie the user actually rated
print("\nHit Rate: ", hitRate(topNPredicted, leftOutPredictions))



Hit Rate:  0.03642384105960265


In [ ]:
import gc
del bigTestSet
del train_loocv
del test_loocv
del allPredictions
gc.collect()

119

##### 2.2. Cumulative Hit Rate: 
see how often we recommended a movie the user actually liked

    Hit rate, confined to ratings above a certain threshold. 

In [ ]:
# See how often we recommended a movie the user actually rated
print("\ncHR (Cumulative Hit Rate, rating >= 4): ", CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))



cHR (Cumulative Hit Rate, rating >= 4):  0.0509571313022378


##### 2.3. Average Reciprocal Hit Rank:
    Hit rate that takes the ranking into account.


In [ ]:
# Compute ARHR
print("\nARHR (Average Reciprocal Hit Rank): ", AverageReciprocalHitRank(topNPredicted, leftOutPredictions))


ARHR (Average Reciprocal Hit Rank):  0.012887036161042793


##### 2.4. Coverage:
    What percentage of users have at least one "good" recommendation


In [ ]:
print("\nComputing complete recommendations, no hold outs...")
svd.fit(fullTrainSet)
bigTestSet = fullTrainSet.build_anti_testset()
allPredictions = svd.test(bigTestSet)
topNPredicted = getTopN(allPredictions, n=10)


Computing complete recommendations, no hold outs...


In [ ]:
# Print user coverage with a minimum predicted rating of 4.0:
print("\nUser coverage: ", UserCoverage(topNPredicted, fullTrainSet.n_users, ratingThreshold=4.0))



User coverage:  0.9897350993377484


##### 2.5. Diversity

In [ ]:
# Measure diversity of recommendations:
print("\nDiversity: ", Diversity(topNPredicted, simsAlgo))

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Diversity:  0.933834367587023


##### 2.6. Novelty

In [ ]:
# Measure novelty (average popularity rank of recommendations):
print("\nNovelty (average popularity rank): ", Novelty(topNPredicted, rankings))


Novelty (average popularity rank):  643.175474968562


In [ ]:
import gc
del bigTestSet
del allPredictions
del topNPredicted
gc.collect()

696

# Summary

The results showed that SVC++ is a bit better that SVD in terms of accuracy and in terms of quality of recommendations as well.
I'am pretty sure we could get better results with some hyperparameters tuning, but SVD and SVD++ can be time-consuming and computationally expensive on a large datasets (as movielens-20m), let alone evaluating all the metrics discussed above.

Here are some downsides of SVD and SVD++ i've found:

* SVD and SVD++ can be computationally expensive: The calculation of SVD and SVD++ involves computing a large number of matrix operations, which can be time-consuming and computationally expensive, especially for large matrices.

* SVD and SVD++ may not work well with sparse matrices: Both SVD and SVD++ assume that the input matrix is dense and complete, which means that there are no missing values. In real-world applications, however, input matrices are often sparse, and SVD and SVD++ may not work well with incomplete data.

* SVD and SVD++ can suffer from overfitting: SVD and SVD++ are prone to overfitting, especially when the input matrix is noisy or when the number of latent factors is too high. This can result in poor performance on unseen data.

* SVD and SVD++ may not capture non-linear relationships: Both SVD and SVD++ assume that the relationships between the items and users are linear. However, in many real-world scenarios, the relationships may be non-linear, and SVD and SVD++ may not be able to capture them effectively.

* SVD++ requires additional user and item features: SVD++ requires additional user and item features, which may not always be available or may be difficult to obtain. Without these features, the performance of SVD++ can be limited.